In [10]:
from dotenv import load_dotenv
from llama_index.llms.openrouter import OpenRouter
from llama_index.core.llms import ChatMessage
from os import getenv

load_dotenv()

/home/tphung/projects/lawrag/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [11]:
llm = OpenRouter(
    max_tokens=256,
    context_window=4096,
    model="cohere/command-r"
)

In [12]:
message = ChatMessage(role="user", content="Tell me a joke")
resp = llm.chat([message])
print(resp)

assistant: What do you call a factory that makes distinctly average products?

*An adequate-ory!*


In [13]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader



In [14]:
documents = SimpleDirectoryReader("poems_data").load_data()

# Trying out Cohere embeddings

In [15]:
from llama_index.embeddings.cohere import CohereEmbedding

embed_model = CohereEmbedding(
    api_key=getenv("COHERE_API_KEY"),
    model_name="embed-multilingual-light-v3.0",
    input_type="search_query",
)

In [16]:
from llama_index.llms.cohere import Cohere

llm_original = Cohere(model="command-r", api_key=getenv("COHERE_API_KEY"))

In [17]:
llm.chat([ChatMessage(role="user", content="Rancontrez-moi une blague")])

ChatResponse(message=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='Une blague pour vous !\n\nDeux atomes hydrogenes se réunissent dans un bar. Le premier hydrogène dit à l\'autre: "j\'ai perdu mon electron."\n\nL\'autre demande: "Vraiment? Comment ça s\'est passé?"\n\nLe premier répond: "Je ne suis pas sûr, ça s\'est passé tellement vite!"\n\nEst-ce que cela vous a fait rire?', additional_kwargs={}), raw=ChatCompletion(id='gen-Rh4ZBrrpzaaDSE7El5mbHEujVQ0E', choices=[Choice(finish_reason='COMPLETE', index=0, logprobs=None, message=ChatCompletionMessage(content='Une blague pour vous !\n\nDeux atomes hydrogenes se réunissent dans un bar. Le premier hydrogène dit à l\'autre: "j\'ai perdu mon electron."\n\nL\'autre demande: "Vraiment? Comment ça s\'est passé?"\n\nLe premier répond: "Je ne suis pas sûr, ça s\'est passé tellement vite!"\n\nEst-ce que cela vous a fait rire?', role='assistant', function_call=None, tool_calls=None))], created=1722938026, model='cohere/command-r

In [18]:
from llama_index.core import Settings

Settings.llm=llm
Settings.embed_model=embed_model

In [19]:
index = VectorStoreIndex.from_documents(
    documents=documents,
)

In [20]:
from llama_index.postprocessor.cohere_rerank import CohereRerank

cohere_rerank = CohereRerank()

In [21]:
query_engine = index.as_query_engine(
    postprocessors=[cohere_rerank],
)

In [22]:
response = query_engine.query("A qui semble le poete?")

In [23]:
response

Response(response='Le Poète semble au prince des nuées.', source_nodes=[NodeWithScore(node=TextNode(id_='1ac37012-aaaf-479f-af51-b7e962afdcd6', embedding=None, metadata={'file_path': '/home/tphung/projects/lawrag/poems_data/albatros.txt', 'file_name': 'albatros.txt', 'file_type': 'text/plain', 'file_size': 762, 'creation_date': '2024-08-05', 'last_modified_date': '2024-08-05'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='f7eda0f4-8215-4846-9f7a-013d728bdd03', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': '/home/tphung/projects/lawrag/poems_data/albatros.txt', 'file_name': 'albatros.txt', 'file_type': 'text/plain', 'file_size': 762, 'creation_date': '2024-08-05', 'last_modified_date': '202

# example of Vectore storage persisting

In [24]:
index.storage_context.persist(persist_dir="./poems_storage") # save the index to ./storage directory

In [25]:
from llama_index.core import load_index_from_storage, StorageContext
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core.storage.index_store import SimpleIndexStore
from llama_index.core.vector_stores import SimpleVectorStore

storage_context = StorageContext.from_defaults(
    persist_dir="poems_storage"
    # theese arguments do not work but they were in guide
    # docstore=SimpleDocumentStore.from_persist_dir(persist_dir="storage"),
    # index_store=SimpleIndexStore.from_persist_dir(persist_dir="storage"),

    # vector_store=SimpleVectorStore.from_persist_dir(persist_dir="storage"),
)
persistent_index = load_index_from_storage(storage_context=storage_context)

In [26]:
persistent_engine = persistent_index.as_query_engine(
    postprocessors=[cohere_rerank],
)

In [27]:
response_new = persistent_engine.query("A qui semble le poete?")
response_new

Response(response='Le Poète semble au prince des nuées.', source_nodes=[NodeWithScore(node=TextNode(id_='1ac37012-aaaf-479f-af51-b7e962afdcd6', embedding=None, metadata={'file_path': '/home/tphung/projects/lawrag/poems_data/albatros.txt', 'file_name': 'albatros.txt', 'file_type': 'text/plain', 'file_size': 762, 'creation_date': '2024-08-05', 'last_modified_date': '2024-08-05'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='f7eda0f4-8215-4846-9f7a-013d728bdd03', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': '/home/tphung/projects/lawrag/poems_data/albatros.txt', 'file_name': 'albatros.txt', 'file_type': 'text/plain', 'file_size': 762, 'creation_date': '2024-08-05', 'last_modified_date': '202

# CSV file as data source

In [32]:
from llama_index.readers.file import PandasCSVReader

# CSVReader does not work properly, so i use PandasCSVReader
reader = PandasCSVReader(concat_rows=False)
travail_documents = reader.load_data(file = "./code_data/extrait_travail.csv")


In [33]:
travail_index =VectorStoreIndex.from_documents(
    documents=travail_documents,
)

In [34]:
travail_query_engine = travail_index.as_query_engine(
    postprocessors=[cohere_rerank],
)

In [35]:
travail_query_engine.query("Quel est le but du travail?")

Response(response="Le travail a pour objectif de présenter les orientations de la politique du gouvernement dans les domaines des relations individuelles et collectives du travail, de l'emploi et de la formation professionnelle.", source_nodes=[NodeWithScore(node=TextNode(id_='a37dffe3-72dc-4299-845d-a0fff431eb03', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='f278a54e-3f77-457c-b13b-7011aedf14d6', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='6542d9b89b4e9959d6f2bbce42aee0388a635d1af2faeb8f4965d758737df357')}, text='LEGIARTI000017961625, LEGITEXT000006072050, L2, ["Partie législative", "Chapitre préliminaire : Dialogue social."], <p>Le Gouvernement soumet les projets de textes législatifs et réglementaires élaborés dans le champ défini par l\'article <a href=\'/affichCodeArticle.do?cidTexte=LEGITEXT000006072050&idArticle=LEGIARTI000017961623&dateTexte=&cate